## Close Source Modelling with BBC News Dataset

For comparison of Summary in different aspects/topics.

Understand LLM Strengths and Weaknesses:

- **Identify domain-specific strengths**: Different LLMs are trained on different datasets and may excel in different domains. Comparing summaries across topics can help you identify which LLM performs best in a specific area relevant to your needs.

- **Uncover biases and limitations**: LLMs can inherit biases from their training data. Comparing summaries can help you identify potential biases and limitations in different models, allowing you to choose the one with the least bias for your task.

- **Evaluate factual accuracy**: Some LLMs prioritize fluency over factual accuracy, while others excel at fact-checking. Comparing summaries can help you assess the factual accuracy of each LLM and choose the one that best suits your need for reliable information.

Steps:

1. **Install & Import Necessary Libraries**


2. **Load Dataset (BBC News)** : Pick 5 rows of data per aspects


3. **OpenAI Topic Summary Generation**


4. **Google Topic Summary Generation**

#### Install & Import Necessary Libraries

In [2]:
import os
import platform
import subprocess
import time
import sys

import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')

sys.path.append('../')
from helper.SummarizationMetrics import SummarizationMetrics
from helper.chatgpt_automation import ChatGPTAutomation, split_text_into_chunks
from helper.bard_automation import BardAutomation, split_text_into_chunks


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from langchain import LLMChain, HuggingFacePipeline

from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import ChatVertexAI
from langchain.llms import VertexAI
import google.generativeai as genai

from sentence_transformers import SentenceTransformer, util
from scipy.signal import argrelextrema
from sklearn.cluster import KMeans

from os import environ

from dotenv import load_dotenv
import pandas as pd
import sys
sys.path.append('../')
from helper.SummarizationMetrics import SummarizationMetrics

[nltk_data] Downloading package punkt to C:\Users\Zhang
[nltk_data]     Xiang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Zhang
[nltk_data]     Xiang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\Zhang Xiang\Desktop\Year 3\Sem 2\FYPJ\Project\RD_TextSummarizerForWebinar\Development\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Load Dataset (BBC)

In [2]:
bbc_train_df = pd.read_excel("../Data/newsbbc_train.xlsx")

bbc_train_df.head()

,File_path,Articles,Summaries,transcript,summary
0,business,Cuba winds back economic clock..Fidel Castro's...,Fidel Castro's decision to ban all cash transa...,Cuba winds back economic clock..Fidel Castro's...,Fidel Castro's decision to ban all cash transa...
1,politics,Blair looks to election campaign..Tony Blair's...,There was little in terms of concrete proposal...,Blair looks to election campaign..Tony Blair's...,There was little in terms of concrete proposal...
2,entertainment,New York rockers top talent poll..New York ele...,New York electro-rock group The Bravery have c...,New York rockers top talent poll..New York ele...,New York electro-rock group The Bravery have c...
3,politics,Terror suspects face house arrest..UK citizens...,British citizens are being included in the cha...,Terror suspects face house arrest..UK citizens...,British citizens are being included in the cha...
4,politics,'No more concessions' on terror..Charles Clark...,"On Monday, MPs voted 272-219 in favour of the ...",'No more concessions' on terror..Charles Clark...,"On Monday, MPs voted 272-219 in favour of the ..."


In [3]:
bbc_train_df = bbc_train_df.groupby('File_path').head(5)
bbc_train_df['File_path'].value_counts()

File_path
business         5
politics         5
entertainment    5
sport            5
tech             5
Name: count, dtype: int64

In [4]:
bbc_train_df

,File_path,Articles,Summaries,transcript,summary
0,business,Cuba winds back economic clock..Fidel Castro's...,Fidel Castro's decision to ban all cash transa...,Cuba winds back economic clock..Fidel Castro's...,Fidel Castro's decision to ban all cash transa...
1,politics,Blair looks to election campaign..Tony Blair's...,There was little in terms of concrete proposal...,Blair looks to election campaign..Tony Blair's...,There was little in terms of concrete proposal...
2,entertainment,New York rockers top talent poll..New York ele...,New York electro-rock group The Bravery have c...,New York rockers top talent poll..New York ele...,New York electro-rock group The Bravery have c...
3,politics,Terror suspects face house arrest..UK citizens...,British citizens are being included in the cha...,Terror suspects face house arrest..UK citizens...,British citizens are being included in the cha...
4,politics,'No more concessions' on terror..Charles Clark...,"On Monday, MPs voted 272-219 in favour of the ...",'No more concessions' on terror..Charles Clark...,"On Monday, MPs voted 272-219 in favour of the ..."
5,politics,Howard denies split over ID cards..Michael How...,Michael Howard has denied his shadow cabinet w...,Howard denies split over ID cards..Michael How...,Michael Howard has denied his shadow cabinet w...
6,sport,Chelsea denied by James heroics..A brave defen...,Chelsea were now looking more like Premiership...,Chelsea denied by James heroics..A brave defen...,Chelsea were now looking more like Premiership...
7,politics,Guantanamo man 'suing government'..A British t...,He said he was sent there after being interrog...,Guantanamo man 'suing government'..A British t...,He said he was sent there after being interrog...
13,business,Could Yukos be a blessing in disguise?..Other ...,But it argues that more rigorous tax policing ...,Could Yukos be a blessing in disguise?..Other ...,But it argues that more rigorous tax policing ...
14,business,Asian quake hits European shares..Shares in Eu...,The unfolding scale of the disaster in south A...,Asian quake hits European shares..Shares in Eu...,The unfolding scale of the disaster in south A...


In [5]:
# Load File that contain API Key
load_dotenv(".env", override=True)

True

In [6]:
# Retrieve API Key
openai_api_key = environ.get("OPENAI_API_KEY")
gcp_api_key = environ.get("GCP_API_KEY")


In [7]:
genai.configure(api_key=gcp_api_key)

In [8]:
# List Model Available on genai for Google.
for model in genai.list_models():
    print(model)

Model(name='models/chat-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 Chat (Legacy)',
      description='A legacy text-only model optimized for chat conversations',
      input_token_limit=4096,
      output_token_limit=1024,
      supported_generation_methods=['generateMessage', 'countMessageTokens'],
      temperature=0.25,
      top_p=0.95,
      top_k=40)
Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      top_p=0.95,
      top_k=40)
Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko',
      description='Obtain a distributed representatio

### OpenAI Topic Summary Generation

In [14]:
# Initialize Models
# Test 1-2 models at a time to avoid Kernel stopping.
openai_models = {
    "models": [
        # "gpt-3.5-turbo-1106",
        # "gpt-3.5-turbo-0125",
        # "gpt-4",
        # "gpt-4-1106-preview",
        "gpt-4-0125-preview",

    ],
    "max_tokens": [
        # 16385,
        # 16385,
        # 8192,
        # 128000,
        128000,
    ]
}

df_openai_models = pd.DataFrame(openai_models)
df_openai_models

,models,max_tokens
0,gpt-4-0125-preview,128000


In [5]:
df_scores = pd.DataFrame(columns=['model', 'method', 'max_tokens', 'topic' ,'num_tokens' ,'transcript', 'original summary' ,'summary', 'grammar', 'readability', 'rouge', 'bert_score', 'time_taken', 'prompt', 'temperature'])
df_scores

# Run Read Excel when adding more data into the DataFrame
df_scores = pd.read_excel("./result/closed_source_model_topics_comparison.xlsx")
df_scores.head(0)

,model,method,max_tokens,topic,num_tokens,transcript,original summary,summary,grammar,readability,rouge,bert_score,time_taken,prompt,temperature,bleu


In [16]:
for model_index, model_row in df_openai_models.iterrows():
    model_name = model_row["models"]
    print(model_name)

    temperature = 0

    llm = ChatOpenAI(temperature=temperature, model_name=model_name, openai_api_key=openai_api_key)
    
    prompt_template = """Write a concise summary of the following: {text}"""
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])

    for index, row in bbc_train_df.iterrows():
        method = "MapReduce"

        # get the summary
        start_time = time.time()
        num_tokens = llm.get_num_tokens(row['transcript'])
        print("Number of tokens:", num_tokens)

        max_tokens = model_row["max_tokens"]

        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=max_tokens-100, chunk_overlap=100)
        docs = text_splitter.create_documents([row['transcript']])
        print("Number of chunks:", len(docs))

        # break
        summary_chain = load_summarize_chain(llm=llm, chain_type='map_reduce', token_max=max_tokens , map_prompt=PROMPT)
        summary = summary_chain.run(docs)

        end_time = time.time()
        elapsed_time = end_time - start_time

        metrics = SummarizationMetrics(row['summary'], summary)

        new_result = {
            'model': model_name,
            'method': method,
            'max_tokens': max_tokens,
            'topic': row["File_path"],
            'transcript': row['transcript'],
            'original summary': row['summary'],
            'summary': summary,
            'rouge': metrics.rouge_scores(),
            'bert_score': metrics.bert_score(),
            'bleu': metrics.bleu_score(),
            'time_taken': elapsed_time,
            'grammar': metrics.grammar_check(),
            'readability': metrics.readability_index(),
            'num_tokens': num_tokens,
            'prompt': prompt_template,
            'temperature': temperature
        }


        new_row = pd.DataFrame([new_result])

        df_scores = pd.concat([df_scores, new_row], ignore_index=True)

gpt-4-0125-preview
Number of tokens: 762
Number of chunks: 1


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:02<00:00,  2.20s/it]


computing greedy matching.


...

calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:01<00:00,  1.66s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 398.70it/s]


done in 1.68 seconds, 0.60 sentences/sec


In [17]:
df_scores

,model,method,max_tokens,topic,num_tokens,transcript,original summary,summary,grammar,readability,rouge,bert_score,time_taken,prompt,temperature,bleu
0,gpt-3.5-turbo-0125,MapReduce,16385,business,762,Cuba winds back economic clock..Fidel Castro's...,Fidel Castro's decision to ban all cash transa...,Cuba has imposed a 10% tax on conversions betw...,[],100 words required.,"[{'rouge-1': {'r': 0.17006802721088435, 'p': 0...","(tensor([0.9130]), tensor([0.8402]), tensor([0...",3.803998,Write a concise summary of the following: {text},0,0.016126
1,gpt-3.5-turbo-0125,MapReduce,16385,politics,702,Blair looks to election campaign..Tony Blair's...,There was little in terms of concrete proposal...,Tony Blair's recent speech marked the start of...,"[Match({'ruleId': 'MORFOLOGIK_RULE_EN_US', 'me...",100 words required.,"[{'rouge-1': {'r': 0.17365269461077845, 'p': 0...","(tensor([0.8809]), tensor([0.8259]), tensor([0...",4.186849,Write a concise summary of the following: {text},0,0.016755
2,gpt-3.5-turbo-0125,MapReduce,16385,entertainment,718,New York rockers top talent poll..New York ele...,New York electro-rock group The Bravery have c...,"The Bravery, a New York electro-rock group, to...","[Match({'ruleId': 'MORFOLOGIK_RULE_EN_US', 'me...",100 words required.,"[{'rouge-1': {'r': 0.26666666666666666, 'p': 0...","(tensor([0.9043]), tensor([0.8542]), tensor([0...",2.662916,Write a concise summary of the following: {text},0,0.057747
3,gpt-3.5-turbo-0125,MapReduce,16385,politics,671,Terror suspects face house arrest..UK citizens...,British citizens are being included in the cha...,The UK government is considering new measures ...,[],100 words required.,"[{'rouge-1': {'r': 0.16265060240963855, 'p': 0...","(tensor([0.8905]), tensor([0.8307]), tensor([0...",3.348620,Write a concise summary of the following: {text},0,0.005401
4,gpt-3.5-turbo-0125,MapReduce,16385,politics,642,'No more concessions' on terror..Charles Clark...,"On Monday, MPs voted 272-219 in favour of the ...",Charles Clarke is standing firm on his anti-te...,"[Match({'ruleId': 'MORFOLOGIK_RULE_EN_US', 'me...",100 words required.,"[{'rouge-1': {'r': 0.2751677852348993, 'p': 0....","(tensor([0.8944]), tensor([0.8478]), tensor([0...",2.810315,Write a concise summary of the following: {text},0,0.061109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,gpt-4-0125-preview,MapReduce,128000,sport,805,Paris promise raises Welsh hopes..Has there be...,But since they threw off the shackles against ...,The article highlights the rising expectations...,[],"score: 14.651290322580646, grade_level: '15'","[{'rouge-1': {'r': 0.17391304347826086, 'p': 0...","([tensor(0.8637)], [tensor(0.8236)], [tensor(0...",33.090430,Write a concise summary of the following: {text},0,0.035691
71,gpt-4-0125-preview,MapReduce,128000,entertainment,707,Rapper Kanye West's shrewd soul..US hip-hop st...,Leaving his Chicago art school after only one ...,"Kanye West, a key figure in hip-hop, leads the...","[Offset 557, length 4, Rule ID: MORFOLOGIK_RUL...","score: 13.59773553719008, grade_level: '14'","[{'rouge-1': {'r': 0.2430939226519337, 'p': 0....","([tensor(0.8815)], [tensor(0.8397)], [tensor(0...",22.018759,Write a concise summary of the following: {text},0,0.034853
72,gpt-4-0125-preview,MapReduce,128000,entertainment,636,Redford's vision of Sundance..Despite sporting...,Redford wanted Sundance to be a platform for i...,Robert Redford founded the Sundance Film Festi...,"[Offset 27, length 8, Rule ID: MORFOLOGIK_RULE...","score: 15.02, grade_level: '15'","[{'rouge-1': {'r': 0.16129032258064516, 'p': 0...","([tensor(0.8860)], [tensor(0.8411)], [tensor(0...",25.070732,Write a concise summary of the following: {text},0,0.025239
73,gpt-4-0125-preview,MapReduce,128000,business,697,Water firm Suez in Argentina row..A conflict b...,The government has rejected the 60% rise and w...,The Argentine government is in a dispute with ...,"[Offset 46, length 5, Rule ID: MORFOLOGIK_RULE...","score: 17.7248333333

In [18]:
# df_scores.to_excel("./result/closed_source_model_topics_comparison.xlsx", index=False)

### Google Topic Summary Generation

In [11]:
df_scores = pd.read_excel("./result/closed_source_model_topics_comparison.xlsx")
print(df_scores.shape)
df_scores.head(0)

(150, 16)


,model,method,max_tokens,topic,num_tokens,transcript,original summary,summary,grammar,readability,rouge,bert_score,time_taken,prompt,temperature,bleu


In [21]:
# Initialize Models
gcp_models = {

    # Models selected
    "models": [
        # "gemini-pro",
        # "chat-bison-001",
        "text-bison-001"
    ],

    # Input Token
    "max_tokens": [
        # 30720,
        # 4096,
        8196,

    ]
}


df_gcp_models = pd.DataFrame(gcp_models)
df_gcp_models

,models,max_tokens
0,text-bison-001,8196


In [22]:
for model_index, model_row in df_gcp_models.iterrows():
    model_name = model_row["models"]
    print(model_name)

    temperature = 0

    if "chat" in model_name.lower():
      llm = ChatVertexAI(temperature=temperature, model=model_name, verbose=True)
    else:
      llm = VertexAI(temperature=temperature, model=model_name, verbose=True)
    
    prompt_template = """Write a concise summary of the following: {text}"""
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])

    for index, row in bbc_train_df.iterrows():
        method = "MapReduce"

        # get the summary
        start_time = time.time()
        num_tokens = llm.get_num_tokens(row['transcript'])
        print("Number of tokens:", num_tokens)

        max_tokens = model_row["max_tokens"]
        print(max_tokens)

        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=max_tokens-100, chunk_overlap=100)
        docs = text_splitter.create_documents([row['transcript']])
        print("Number of chunks:", len(docs))

        summary_chain = load_summarize_chain(llm=llm, chain_type='map_reduce', token_max=max_tokens , map_prompt=PROMPT)
        summary = summary_chain.run(docs)

        end_time = time.time()
        elapsed_time = end_time - start_time

        metrics = SummarizationMetrics(row['summary'], summary)

        new_result = {
            'model': model_name,
            'method': method,
            'max_tokens': max_tokens,
            'topic': row["File_path"],
            'transcript': row['transcript'],
            'original summary': row['summary'],
            'summary': summary,
            'rouge': metrics.rouge_scores(),
            'bert_score': metrics.bert_score(),
            'bleu': metrics.bleu_score(),
            'time_taken': elapsed_time,
            'grammar': metrics.grammar_check(),
            'readability': metrics.readability_index(),
            'num_tokens': num_tokens,
            'prompt': prompt_template,
            'temperature': temperature
        }


        new_row = pd.DataFrame([new_result])

        df_scores = pd.concat([df_scores, new_row], ignore_index=True)


text-bison-001
Number of tokens: 778
8196
Number of chunks: 1


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:02<00:00,  2.04s/it]


computing greedy matching.


...

calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:02<00:00,  2.09s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 97.50it/s]


done in 2.12 seconds, 0.47 sentences/sec


In [14]:
df_scores.to_excel("./result/closed_source_model_topics_comparison.xlsx", index=False)